In [25]:
import requests
import pandas as pd
import numpy as np
import tensorflow_hub as hub


In [4]:
# Load the pre-trained Universal Sentence Encoder model
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


In [5]:
df = pd.read_csv("cancer_diagnosis_clean.csv")
df

,Patient_ID,Age,Gender,Tumor_Size,Tumor_Type,Biopsy_Result,Treatment,Response_to_Treatment,Survival_Status
0,c044501a-43ca-4a0c-8b8b-991439ba1b6a,52,Female,5.08,Benign,Positive,Surgery,No Response,Survived
1,b8900c4c-1232-4084-9432-5d02eba74d20,32,Female,0.80,Benign,Negative,Surgery,Complete Response,Survived
2,3004e2bc-8037-49cb-a542-d5612b73beab,70,Female,9.56,Benign,Positive,Radiation Therapy,Complete Response,Deceased
3,1df86af7-6745-4dea-b127-cbc9915079fc,21,Female,3.07,Malignant,Negative,Surgery,Partial Response,Survived
4,128e00c3-72e3-4031-a7f4-1165d7199cce,62,Male,7.17,Malignant,Positive,Radiation Therapy,Complete Response,Deceased
...,...,...,...,...,...,...,...,...,...
19995,863e873e-2934-4514-94e1-a2ce6429dddd,71,Female,8.99,Benign,Negative,Radiation Therapy,Partial Response,Deceased
19996,854c96f3-39eb-40bb-b50d-0640a356549a,35,Male,4.25,Benign,Negative,Chemotherapy,Complete Response,Survived
19997,8c1f285d-48bf-4cfa-9a89-da083b7943f2,56,Female,6.43,Benign,Negative,Radiation Therapy,Partial Response,Survived
19998,c6dc70ee-4dcd-41e6-aae6-043923cb12c0,39,Female,6.08,Malignant,Negative,Radiation Therapy,No Response,Survived


In [6]:
ages = sorted(df['Age'].unique())
genders = sorted(df['Gender'].unique())
tumor_sizes = sorted(df['Tumor_Size'].unique())
biopsy_results = sorted(df['Biopsy_Result'].unique())
treatments = (df['Treatment'].unique())
treatment_responses = (df['Response_to_Treatment'].unique())
survival_status = (df['Survival_Status'].unique())

print(f"Ages                 : {ages[0]} to {ages[len(ages)-1]}")
print(f"Genders              : {genders}")
print(f"Tumor Sizes          : {tumor_sizes[0]} to {tumor_sizes[len(tumor_sizes)-1]}")
print(f"Results              : {biopsy_results}")
print(f"Treatments           : {treatments}")
print(f"Treatment Responses  : {treatment_responses}")
print(f"Survival Statuses    : {survival_status}")

Ages                 : 20 to 80
Genders              : ['Female', 'Male']
Tumor Sizes          : 0.5 to 10.0
Results              : ['Negative', 'Positive']
Treatments           : ['Surgery' 'Radiation Therapy' 'Chemotherapy']
Treatment Responses  : ['No Response' 'Complete Response' 'Partial Response']
Survival Statuses    : ['Survived' 'Deceased']


# Do Process

In [7]:
df.count()

Patient_ID               20000
Age                      20000
Gender                   20000
Tumor_Size               20000
Tumor_Type               20000
Biopsy_Result            20000
Treatment                20000
Response_to_Treatment    20000
Survival_Status          20000
dtype: int64

In [8]:
def prompt(query):
    return f"""
I have Pandas Dataframe with following column:
* Age: contain real values from 20 to 80
* Gender: contain enum string "Female" or "Male" only
* Tumor_Size: contain floating values ranging from 0.5 to 10.0
* Tumor_Type: contain enum string "Benign" or "Malignant" only
* Biopsy_Result: contain enum string "Negative" or "Positive"
* Treatment: contain enum string "Chemotherapy", "Radiation Therapy", or "Surgery"
* Response_to_Treatment: contain enum values "Complete Response", "No Response", or "Partial Response"
* Survival_Status: contain enum string "Deceased" or "Survived" only

Please create one-line Pandas Dataframe query with consistent code without any explanation (just write one-line code without backtick and use variable `result` as output) if I have variable df that have all the data above if user ask this query:
    
{query}
    """

In [9]:
# user_query = "how many patients with age ranging from 40-60 with biopsy result positive?"
# user_query = "distribution of tumor size on female patient with Malignant"
user_query = "age between 50 until 60 and female and positif and benign"

In [10]:
def query_llama(query):
    # Define the API endpoint
    url = "http://localhost:11434/api/generate"


    # Define the payload
    payload = {
        "model": "llama3.1",
        "stream": False,
        "prompt": prompt(query),
    }

    # Define headers, including the authorization token if required
    headers = {
        "Content-Type": "application/json",
    }

    # Send the request
    response = requests.post(url, json=payload, headers=headers)

    # Check the response
    if response.status_code == 200:
        # Handle the response from the API
        code = response.json()['response']
        new_code = ""
        for row in code.split('\n'):
            if "```" in row:
                continue
            new_code += row

        if new_code != "":
            code = new_code

        return code

    else:
        raise Exception(f"Request failed with status code {response.status_code}: {response.text}")


In [11]:
generated_q = query_llama(user_query)
print(generated_q)

`result = df[(df['Age'] >= 50) & (df['Age'] <= 60) & (df['Gender'] == 'Female') & (df['Biopsy_Result'] == 'Positive') & (df['Tumor_Type'] == 'Benign')]`


In [24]:
model([generated_q]).shape

TensorShape([1, 512])

In [12]:
df[(df['Age'] >= 50) & (df['Age'] <= 60) & (df['Gender'] == 'Female') & (df['Biopsy_Result'] == 'Positive') & (df['Tumor_Type'] == 'Benign')]

,Patient_ID,Age,Gender,Tumor_Size,Tumor_Type,Biopsy_Result,Treatment,Response_to_Treatment,Survival_Status
0,c044501a-43ca-4a0c-8b8b-991439ba1b6a,52,Female,5.08,Benign,Positive,Surgery,No Response,Survived
11,4e9ae3b3-f55c-409e-8d81-79051572acf1,55,Female,0.81,Benign,Positive,Surgery,No Response,Deceased
102,b3e3e50b-1328-499b-9fac-ac4008cb8365,52,Female,0.89,Benign,Positive,Surgery,Partial Response,Deceased
188,8d54fea3-f739-4165-8d77-9a6fc668e48f,53,Female,1.84,Benign,Positive,Radiation Therapy,No Response,Deceased
202,bb5a381b-399f-4e1c-8ac0-7fd810b224f7,59,Female,4.95,Benign,Positive,Radiation Therapy,No Response,Deceased
...,...,...,...,...,...,...,...,...,...
19577,db7e0519-1a7e-4b4d-80df-b83039caa93e,59,Female,0.60,Benign,Positive,Chemotherapy,No Response,Deceased
19603,748bef26-ce3c-47d8-a3a5-62cf54ba91c4,52,Female,9.59,Benign,Positive,Chemotherapy,No Response,Survived
19646,d069e1e6-4920-4f3b-b2f8-517e9b307998,51,Female,5.79,Benign,Positive,Radiation Therapy,Complete Response,Deceased
19833,7b2c9233-609d-4bb4-8321-a395b868fadb,57,Female,1.83,Benign,Positive,Chemotherapy,No Response,Survived


In [13]:
for i in range(0, 5):
    generated_q = query_llama(user_query)
    print(generated_q)

result = df[(df['Age'] >= 50) & (df['Age'] <= 60) & (df['Gender'] == 'Female') & (df['Biopsy_Result'] == 'Positive') & (df['Tumor_Type'] == 'Benign')]
result = df[(df['Age'] >= 50) & (df['Age'] <= 60) & (df['Gender'] == 'Female') & (df['Biopsy_Result'] == 'Positive') & (df['Tumor_Type'] == 'Benign')]
result = df[(df['Age'] >= 50) & (df['Age'] <= 60) & (df['Gender'] == 'Female') & (df['Biopsy_Result'] == 'Positive') & (df['Tumor_Type'] == 'Benign')]
result = df[(df['Age'] >= 50) & (df['Age'] <= 60) & (df['Gender'] == 'Female') & (df['Biopsy_Result'] == 'Positive') & (df['Tumor_Type'] == 'Benign')]
result = df[(df['Age'] >= 50) & (df['Age'] <= 60) & (df['Gender'] == 'Female') & (df['Biopsy_Result'] == 'Positive') & (df['Tumor_Type'] == 'Benign')]


In [14]:
local_vars = {}
exec(generated_q, globals(), local_vars)
print(local_vars['result'])

                                 Patient_ID  Age  Gender  Tumor_Size  \
0      c044501a-43ca-4a0c-8b8b-991439ba1b6a   52  Female        5.08   
11     4e9ae3b3-f55c-409e-8d81-79051572acf1   55  Female        0.81   
102    b3e3e50b-1328-499b-9fac-ac4008cb8365   52  Female        0.89   
188    8d54fea3-f739-4165-8d77-9a6fc668e48f   53  Female        1.84   
202    bb5a381b-399f-4e1c-8ac0-7fd810b224f7   59  Female        4.95   
...                                     ...  ...     ...         ...   
19577  db7e0519-1a7e-4b4d-80df-b83039caa93e   59  Female        0.60   
19603  748bef26-ce3c-47d8-a3a5-62cf54ba91c4   52  Female        9.59   
19646  d069e1e6-4920-4f3b-b2f8-517e9b307998   51  Female        5.79   
19833  7b2c9233-609d-4bb4-8321-a395b868fadb   57  Female        1.83   
19931  c88faaac-ba83-4a13-890d-e0577e21b89e   50  Female        6.53   

      Tumor_Type Biopsy_Result          Treatment Response_to_Treatment  \
0         Benign      Positive            Surgery           

In [15]:
query_map = [
    {
        "question": "how many patients with age ranging from 40-60 with biopsy result positive?",
        "query": "result = df[(df['Age'] >= 40) & (df['Age'] <= 60) & (df['Biopsy_Result'] == 'Positive')].shape[0]"
    },
    {
        "question": "distribution of tumor size on female patient with Malignant",
       "query": "result = df[(df['Gender'] == 'Female') & (df['Tumor_Type'] == 'Malignant')][['Tumor_Size(cm)']].describe()",
    },
    {
        "question": "What is the median tumor size for patients who underwent surgery?",
        "query": "result = df[df['Treatment'] == 'Surgery']['Tumor_Size'].median()"
    },
    {
        "question": "How many patients over 60 years old have malignant tumors?",
        "query": "result = df[(df['Age'] > 60) & (df['Tumor_Type'] == 'Malignant')].shape[0]"
    },
    {
        "question": "What is the survival rate for patients who received radiation therapy?",
        "query": "result = df[df['Treatment'] == 'Radiation Therapy']['Survival_Status'].value_counts(normalize=True)['Survived'] * 100"
    },
    {
        "question": "What is the average age difference between male and female patients?",
        "query": "result = df.groupby('Gender')['Age'].mean().diff().iloc[-1]"
    },
    {
        "question": "How many patients had a complete response to treatment and survived?",
        "query": "result = df[(df['Response_to_Treatment'] == 'Complete Response') & (df['Survival_Status'] == 'Survived')].shape[0]"
    },
    {
        "question": "What percentage of patients with tumors smaller than 2.0 had benign tumors?",
        "query": "result = df[df['Tumor_Size'] < 2.0]['Tumor_Type'].value_counts(normalize=True)['Benign'] * 100"
    },
    {
        "question": "What is the average tumor size for patients who did not respond to treatment?",
        "query": "result = df[df['Response_to_Treatment'] == 'No Response']['Tumor_Size'].mean()"
    },
    {
        "question": "How many male patients over 70 years old received chemotherapy?",
        "query": "result = df[(df['Gender'] == 'Male') & (df['Age'] > 70) & (df['Treatment'] == 'Chemotherapy')].shape[0]"
    },
    {
        "question": "What is the survival rate for patients with malignant tumors?",
        "query": "result = df[df['Tumor_Type'] == 'Malignant']['Survival_Status'].value_counts(normalize=True)['Survived'] * 100"
    },
    {
        "question": "What is the most common treatment for patients under 40?",
        "query": "result = df[df['Age'] < 40]['Treatment'].mode()[0]"
    },
]

In [16]:
len(query_map)

12

In [31]:
total_similarity = 0

for row in query_map:
    question = row['question']
    query = row['query']

    # Generate embeddings for each sentence
    expected_query_embeddings = model([query])
    actual_generated = query_llama(question)
    actual_query_embeddings = model([actual_generated])

    expected_query_embeddings = expected_query_embeddings.numpy().flatten()
    actual_query_embeddings = actual_query_embeddings.numpy().flatten()

    # Compute the dot product of the vectors
    dot_product = np.dot(expected_query_embeddings, actual_query_embeddings)
    
    # Compute the norms (magnitudes) of the vectors
    norm_a = np.linalg.norm(expected_query_embeddings)
    norm_b = np.linalg.norm(actual_query_embeddings)
    
    # Compute cosine similarity
    cosine_similarity = dot_product / (norm_a * norm_b)
    total_similarity += cosine_similarity
    
    # Convert cosine similarity to percentage
    percentage_similarity = (cosine_similarity + 1) / 2 * 100
    formatted_similarity = "{:.2f}".format(percentage_similarity)
    
    print(f"{question}")
    print(f"Expected          : {query}")
    print(f"Actual            : {actual_generated}")
    print(f"Cosine similarity : {formatted_similarity}%")
    print()
    # local_vars = {}
    # exec(actual_generated, globals(), local_vars)
    # print(local_vars['result'])
    # print("="*100)

how many patients with age ranging from 40-60 with biopsy result positive?
Expected          : result = df[(df['Age'] >= 40) & (df['Age'] <= 60) & (df['Biopsy_Result'] == 'Positive')].shape[0]
Actual            : result = df[(df['Age'] >= 40) & (df['Age'] <= 60) & (df['Biopsy_Result'] == 'Positive')].shape[0]
Cosine similarity : 100.00%

distribution of tumor size on female patient with Malignant
Expected          : result = df[(df['Gender'] == 'Female') & (df['Tumor_Type'] == 'Malignant')][['Tumor_Size(cm)']].describe()
Actual            : result = df[(df['Gender'] == 'Female') & (df['Tumor_Type'] == 'Malignant')][['Tumor_Size']].describe()
Cosine similarity : 97.48%

What is the median tumor size for patients who underwent surgery?
Expected          : result = df[df['Treatment'] == 'Surgery']['Tumor_Size'].median()
Actual            : result = df[df['Treatment'] == 'Surgery']['Tumor_Size'].median()
Cosine similarity : 100.00%

How many patients over 60 years old have malignant tumors

In [34]:
average_similarity = total_similarity / len(query_map)
average_similarity_pct = (average_similarity + 1) / 2 * 100
formatted_avg_sim_pct = "{:.2f}".format(average_similarity_pct)

print(f"Average similarity: {formatted_avg_sim_pct}%")

Average similarity: 95.55%
